Creating a machine learning model to differentiate between two states (camera on and camera off) based on spectrum analyzer data is an interesting problem that can be approached using various techniques. Given the nature of your data (frequency-domain data from a spectrum analyzer), a Convolutional Neural Network (CNN) is well-suited for this task because CNNs are effective in capturing spatial hierarchies and patterns in data.

Here's the approach we'll take:
- Data Preprocessing: Normalize the data to ensure all features contribute equally to the model.
- Data Augmentation: Use techniques like adding Gaussian noise or jitter to increase the amount of training data.
- Model Selection: Use a CNN because it can effectively capture spatial dependencies and patterns in the spectrum data.
- Training and Validation: Split the data into training and validation sets to evaluate the model performance.
- Optimization: Use techniques like early stopping and learning rate reduction on the plateau to optimize the training process.
Let's start with the Python code to preprocess the data, build, and train the CNN model.

In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import tensorflow as tf
import joblib
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, MaxPooling1D, Flatten, Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau

In [2]:
# Load the data
cam_on = pd.read_csv('output_file/clearwrite_captured_SWEEP_REC_2024-06-11 20h09m34s_cam_off.csv', header=None)
cam_on = cam_on.iloc[:, 1:]
cam_off = pd.read_csv('output_file/clearwrite_captured_SWEEP_REC_2024-06-11 21h01m22s_cam_on.csv', header=None)
cam_off = cam_off.iloc[:, 1:]

# Create labels
labels_on = np.ones(cam_on.shape[0])
labels_off = np.zeros(cam_off.shape[0])

# Combine data
data = pd.concat([cam_on, cam_off])
labels = np.concatenate([labels_on, labels_off])

# Normalize the data
scaler = StandardScaler()
data_normalized = scaler.fit_transform(data)
joblib.dump(scaler, 'models/scaler_cnn.pkl')

# Split into training and test sets
X_train, X_test, y_train, y_test = train_test_split(data_normalized, labels, test_size=0.2, random_state=42)

# Reshape data for CNN input
X_train = X_train[..., np.newaxis]
X_test = X_test[..., np.newaxis]


Define the CNN model

In [3]:
# Define the model
model = Sequential([
    Conv1D(filters=64, kernel_size=3, activation='relu', input_shape=(X_train.shape[1], 1)),
    MaxPooling1D(pool_size=2),
    Conv1D(filters=128, kernel_size=3, activation='relu'),
    MaxPooling1D(pool_size=2),
    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(1, activation='sigmoid')
])

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

Training the model

In [4]:
# Define the model
model = Sequential([
    Conv1D(filters=64, kernel_size=3, activation='relu', input_shape=(X_train.shape[1], 1)),
    MaxPooling1D(pool_size=2),
    Conv1D(filters=128, kernel_size=3, activation='relu'),
    MaxPooling1D(pool_size=2),
    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(1, activation='sigmoid')
])

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

Evaluate the model

In [5]:
# Evaluate the model on the test set
test_loss, test_accuracy = model.evaluate(X_test, y_test)
print(f'Test accuracy: {test_accuracy:.4f}')

125/125 [==============================] - 2s 13ms/step - loss: 0.7209 - accuracy: 0.0610
Test accuracy: 0.0610


Save the model

In [6]:
# Save the entire model to a HDF5 file or TensorFlow SavedModel format
model.save('spectrum_analyzer_model.h5')  # Using HDF5 format